In [1]:
import pandas as pd
from src.models.model1.model_preprocess import build_dataset, align_input_to_labels
from src.models.model1.model_preprocess import build_dataset_2


In [2]:
input_group_software_id = pd.read_csv('data/interim/input_group_software_id_vocab.csv', header = None)
input_technique_data_sources = pd.read_csv('data/interim/input_technique_data_sources_vocab.csv', header= None)
input_technique_defenses_bypassed = pd.read_csv('data/interim/input_technique_defenses_bypassed_vocab.csv', header= None)
input_technique_detection_name = pd.read_csv('data/interim/input_technique_detection_name_vocab.csv', header= None)
input_technique_mitigation_id = pd.read_csv('data/interim/input_technique_mitigation_id_vocab.csv', header= None)
input_technique_permissions_required = pd.read_csv('data/interim/input_technique_permissions_required_vocab.csv', header= None)
input_technique_platforms = pd.read_csv('data/interim/input_technique_platforms_vocab.csv', header= None)
input_technique_software_id = pd.read_csv('data/interim/input_technique_software_id_vocab.csv', header= None)
input_technique_tactics = pd.read_csv('data/interim/input_technique_tactics_vocab.csv', header= None)

In [3]:
vocabs = {
    'input_group_software_id' : input_group_software_id[0].values,
    'input_technique_data_sources' : input_technique_data_sources[0].values,
    'input_technique_defenses_bypassed' : input_technique_defenses_bypassed[0].values,
    'input_technique_detection_name' : input_technique_detection_name[0].values,
    'input_technique_mitigation_id' : input_technique_mitigation_id[0].values,
    'input_technique_permissions_required' : input_technique_permissions_required[0].values,
    'input_technique_platforms' : input_technique_platforms[0].values,
    'input_technique_software_id' : input_technique_software_id[0].values,
    'input_technique_tactics' : input_technique_tactics[0].values    
}


In [4]:
group_features_df = pd.read_pickle ('scripts/tmp_selected_group_features.pkl')
technique_features_df = pd.read_pickle ('scripts/tmp_selected_technique_features.pkl')
train_y_df = pd.read_csv ('data/interim/y_cleaned.csv')
train_X_group_df = align_input_to_labels (group_features_df, 
                                            object= 'group', 
                                            label_df= train_y_df)
train_X_technique_df = align_input_to_labels (technique_features_df, 
                                                object= 'technique', 
                                                label_df= train_y_df)

In [5]:
dataset = build_dataset_2 (X_group_df= train_X_group_df, X_technique_df=train_X_technique_df, y_df= train_y_df, ragged_input= True)


In [6]:
dataset.element_spec

({'input_group_software_id': RaggedTensorSpec(TensorShape([None]), tf.string, 0, tf.int64),
  'input_technique_data_sources': RaggedTensorSpec(TensorShape([None]), tf.string, 0, tf.int64),
  'input_technique_defenses_bypassed': RaggedTensorSpec(TensorShape([None]), tf.string, 0, tf.int64),
  'input_technique_detection_name': RaggedTensorSpec(TensorShape([None]), tf.string, 0, tf.int64),
  'input_technique_mitigation_id': RaggedTensorSpec(TensorShape([None]), tf.string, 0, tf.int64),
  'input_technique_permissions_required': RaggedTensorSpec(TensorShape([None]), tf.string, 0, tf.int64),
  'input_technique_platforms': RaggedTensorSpec(TensorShape([None]), tf.string, 0, tf.int64),
  'input_technique_software_id': RaggedTensorSpec(TensorShape([None]), tf.string, 0, tf.int64),
  'input_technique_tactics': RaggedTensorSpec(TensorShape([None]), tf.string, 0, tf.int64)},
 TensorSpec(shape=(), dtype=tf.float32, name=None))

### unit test: `model_v0_5`

In [7]:
from src.models.model1 import model_v0_5

In [8]:
model = model_v0_5.Model1 (
    name = 'model',
    group_nn_hidden_layer_widths =  64,
    group_nn_hidden_layer_depth =  3,
    technique_nn_hidden_layer_widths =  256,
    technique_nn_hidden_layer_depth =  3,
    nn_output_size =  16,
    vocabs= vocabs
)

In [9]:
b_dataset = dataset.batch(32)
model.predict (b_dataset)

1649/1649 [==============================] - 7s 4ms/step


array([[-0.20070034],
       [-0.18215957],
       [-0.21262383],
       ...,
       [-0.10943798],
       [-0.24375904],
       [-0.05363335]], dtype=float32)

In [10]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vectorize_group_software_id  multiple                 0         
  (TextVectorization)                                            
                                                                 
 vectorize_technique_data_so  multiple                 0         
 urces (TextVectorization)                                       
                                                                 
 vectorize_technique_defense  multiple                 0         
 s_bypassed (TextVectorizati                                     
 on)                                                             
                                                                 
 vectorize_technique_detecti  multiple                 0         
 on_name (TextVectorization)                                     
                                                             